In [1]:
from doped.generation import DefectsGenerator
from monty.serialization import loadfn, dumpfn
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet, MPMetalRelaxSet
from pymatgen.io.vasp import Poscar

In [3]:
Al_prim = loadfn('../../../data/matching_AlCr_ox.json')

In [5]:
structure = Al_prim['mp-30084']['structure']
mp_relax_set = MPRelaxSet(structure)
mp_relax_set.write_input("relax_prim_Al2FeO4")

In [7]:
from pymatgen.io.vasp import Poscar
poscar = Poscar.from_file("relax_prim_Al2FeO4/CONTCAR")
relaxed_structure = poscar.structure

In [13]:
defect_gen = DefectsGenerator(
    relaxed_structure, 
    extrinsic=["Cr"],
    interstitial_gen_kwargs=False,
    supercell_gen_kwargs={'min_image_distance':5}
)

Generating DefectEntry objects: 100.0%|████████████████████████████████████████████████████████████| [01:10,   1.43it/s]


Vacancies    Guessed Charges    Conv. Cell Coords    Wyckoff
-----------  -----------------  -------------------  ---------
v_Al         [+1,0,-1,-2,-3]    [0.625,0.625,0.625]  16d
v_Fe         [+1,0,-1,-2]       [0.000,0.000,0.000]  8a
v_O          [+2,+1,0,-1]       [0.391,0.391,0.391]  32e

Substitutions    Guessed Charges     Conv. Cell Coords    Wyckoff
---------------  ------------------  -------------------  ---------
Al_Fe            [+1,0]              [0.000,0.000,0.000]  8a
Al_O             [+5,+4,+3,+2,+1,0]  [0.391,0.391,0.391]  32e
Fe_Al            [+1,0,-1,-2]        [0.625,0.625,0.625]  16d
Fe_O             [+5,+4,+3,+2,+1,0]  [0.391,0.391,0.391]  32e
O_Al             [0,-1,-2,-3,-4,-5]  [0.625,0.625,0.625]  16d
O_Fe             [0,-1,-2,-3,-4]     [0.000,0.000,0.000]  8a
Cr_Al            [+3,+2,+1,0,-1]     [0.625,0.625,0.625]  16d
Cr_Fe            [+4,+3,+2,+1,0]     [0.000,0.000,0.000]  8a
Cr_O             [+5,+4,+3,+2,+1,0]  [0.391,0.391,0.391]  32e

The number in t

In [14]:
defect_gen.defects = {"substitutions": [d for d in defect_gen.defects["substitutions"] if d.name == "Cr_Al"]}
defect_gen.defect_entries = {k: v for k, v in defect_gen.defect_entries.items() if "Cr_Al" in k and k[-1] == "0"}
defect_gen

DefectsGenerator for input composition FeAl2O4, space group Fd-3m with 1 defect entries created.
---------------------------------------------------------
Substitutions    Guessed Charges    Conv. Cell Coords    Wyckoff
---------------  -----------------  -------------------  ---------
Cr_Al            [0]                [0.625,0.625,0.625]  16d

The number in the Wyckoff label is the site multiplicity/degeneracy of that defect in the conventional ('conv.') unit cell, which comprises 8 formula unit(s) of FeAl2O4.

In [15]:
from doped.vasp import DefectsSet

LDAU_PARAMS = {
  "LDAU": True,
    "LDAUJ": {
        "F": {
            "Co": 0,
            "Cr": 0,
            "Fe": 0,
            "Mn": 0,
            "Mo": 0,
            "Ni": 0,
            "V": 0,
            "W": 0
        },
        "O": {
            "Co": 0,
            "Cr": 0,
            "Fe": 0,
            "Mn": 0,
            "Mo": 0,
            "Ni": 0,
            "V": 0,
            "W": 0
        }
    },
    "LDAUL": {
        "F": {
            "Co": 2,
            "Cr": 2,
            "Fe": 2,
            "Mn": 2,
            "Mo": 2,
            "Ni": 2,
            "V": 2,
            "W": 2
        },
        "O": {
            "Co": 2,
            "Cr": 2,
            "Fe": 2,
            "Mn": 2,
            "Mo": 2,
            "Ni": 2,
            "V": 2,
            "W": 2
        }
    },
    "LDAUTYPE": 2,
    "LDAUU": {
        "F": {
            "Co": 3.32,
            "Cr": 3.7,
            "Fe": 5.3,
            "Mn": 3.9,
            "Mo": 4.38,
            "Ni": 6.2,
            "V": 3.25,
            "W": 6.2
        },
        "O": {
            "Co": 3.32,
            "Cr": 3.7,
            "Fe": 5.3,
            "Mn": 3.9,
            "Mo": 4.38,
            "Ni": 6.2,
            "V": 3.25,
            "W": 6.2
        }
    },
    "MAGMOM": {
        "Ce": 5,
        "Ce3+": 1,
        "Co": 0.6,
        "Co3+": 0.6,
        "Co4+": 1,
        "Cr": 5,
        "Dy3+": 5,
        "Er3+": 3,
        "Eu": 10,
        "Eu2+": 7,
        "Eu3+": 6,
        "Fe": 5,
        "Gd3+": 7,
        "Ho3+": 4,
        "La3+": 0.6,
        "Lu3+": 0.6,
        "Mn": 5,
        "Mn3+": 4,
        "Mn4+": 3,
        "Mo": 5,
        "Nd3+": 3,
        "Ni": 5,
        "Pm3+": 4,
        "Pr3+": 2,
        "Sm3+": 5,
        "Tb3+": 6,
        "Tm3+": 2,
        "V": 5,
        "W": 5,
        "Yb3+": 1
    }
}

defect_set = DefectsSet(
    defect_gen,
    user_incar_settings={"ENCUT": 520, "LHFCALC": None, "HFSCREEN": None, "AEXX": None, "GGE":"PBE", **LDAU_PARAMS},
    soc=False,
    user_potcar_functional="PBE",
)
defect_set.write_files("5_doped-defs-Al2FeO4", poscar=True)

vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
vasp.py:229: BadIncarWarning: Cannot find GGE from your user_incar_settings in the list of INCAR flags
Generating and writing input files: 100%|█████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.46s/it]
